## Brain Tumour Detection model using XGBoost Classifier and Hyperparameter Optimization using Raytune:

In [1]:
#pip install ray[tune]

In [2]:
# Importing Ray:
import ray
print('ray %s' % ray.__version__)

ray 1.11.0


In [3]:
# Importing required Libraries:
import os
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback

In [4]:
# Creating a Objective function:
def objective(config: dict):
    
    # Reading the dataset:
    df = pd.read_csv("/home/vivek/Project/new/Zernike_Moments_YN_250.csv", header=None)
    
    # Shuffling the whole dataset:
    df = df.sample(frac=1, random_state=0).reset_index(drop=True)
    
    # Converting categorical values of Target feature into numerical:
    df[289].replace(['YES','NO'], [1,0], inplace=True)
    
    # Independent and Dependent features:
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Train-Test spilt:
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(x_train, label = y_train)
    test_set = xgb.DMatrix(x_test, label = y_test)
    
    # Train the classifier, using the Tune callback
    xgb.train(
              config,
              train_set,
              evals = [(test_set, "eval")],
              verbose_eval = False,
              callbacks = [TuneReportCheckpointCallback(filename="model.xgb")]
              )

def get_best_model_checkpoint(analysis):
    best_bst = xgb.Booster()
    best_bst.load_model(os.path.join(analysis.best_checkpoint, "model.xgb"))
    accuracy = 1. - analysis.best_result["eval-error"]
    
    print(f"Best model parameters: {analysis.best_config}")
    print(f"Best model total accuracy: {accuracy:.4f}")

    return best_bst

def tune_xgboost():

    search_space = {
                    "objective": "binary:logistic",
                    "learning_rate": tune.loguniform(0.0001, 2.0),
                    "eval_metric": ["logloss", "error"],
                    "max_depth": tune.randint(1, 20),
                    "min_child_weight": tune.choice([1,2,3,4,5,6,7,8,9,10])
                    }

    # This will enable aggressive early stopping of bad trials.

    scheduler = ASHAScheduler(
                              max_t = 100,  # 10 training iterations
                              grace_period = 1,
                              reduction_factor = 2
                             )


    analysis = tune.run(
                        objective,
                        metric = "eval-logloss",
                        mode = "min",
                        # You can add "gpu": 0.1 to allocate GPUs
                        resources_per_trial = {"cpu": 1},
                        config = search_space,
                        num_samples = 100,
                        scheduler = scheduler
                       )

    return analysis

if __name__ == "__main__":
    
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument(
                        "--server-address",
                        type=str,
                        default=None,
                        required=False,
                        help="The address of server to connect to if using Ray Client."
                        )
    
    args, _ = parser.parse_known_args()

    if args.server_address:
        
        import ray
        ray.init(f"ray://{args.server_address}")

    analysis = tune_xgboost()

    # Load the best model checkpoint.
    if args.server_address:
        # If connecting to a remote server with Ray Client, checkpoint loading
        # should be wrapped in a task so it will execute on the server.
        # We have to make sure it gets executed on the same node that
        # ``tune.run`` is called on.
        from ray.util.ml_utils.node import force_on_current_node
        remote_fn = force_on_current_node(ray.remote(get_best_model_checkpoint))
        best_bst = ray.get(remote_fn.remote(analysis))
    else:
        best_bst = get_best_model_checkpoint(analysis)

2022-04-08 15:44:38,864	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
/home/vivek/vivek/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trial name,status,loc,learning_rate,max_depth,min_child_weight
objective_b2d55_00000,RUNNING,192.168.29.18:45676,0.00103179,6,8
objective_b2d55_00001,PENDING,,1.82752,5,3
objective_b2d55_00002,PENDING,,0.119756,13,9
objective_b2d55_00003,PENDING,,0.000468887,6,3
objective_b2d55_00004,PENDING,,0.00809948,16,6
objective_b2d55_00005,PENDING,,0.268566,9,10
objective_b2d55_00006,PENDING,,0.230558,6,7
objective_b2d55_00007,PENDING,,1.31755,2,10
objective_b2d55_00008,PENDING,,0.0106242,12,9
objective_b2d55_00009,PENDING,,0.0208905,10,10


Result for objective_b2d55_00000:
  date: 2022-04-08_15-44-40
  done: false
  eval-error: 0.32
  eval-logloss: 0.692898
  experiment_id: 7c45cfc3a45948b1b921d4477ce1c3c2
  hostname: home
  iterations_since_restore: 1
  node_ip: 192.168.29.18
  pid: 45676
  should_checkpoint: true
  time_since_restore: 0.7390120029449463
  time_this_iter_s: 0.7390120029449463
  time_total_s: 0.7390120029449463
  timestamp: 1649412880
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b2d55_00000
  
Result for objective_b2d55_00001:
  date: 2022-04-08_15-44-42
  done: true
  eval-error: 0.34
  eval-logloss: 0.701446
  experiment_id: 2c3fc0057de649ab860de9192cbf46b7
  hostname: home
  iterations_since_restore: 1
  node_ip: 192.168.29.18
  pid: 45674
  should_checkpoint: true
  time_since_restore: 1.7478163242340088
  time_this_iter_s: 1.7478163242340088
  time_total_s: 1.7478163242340088
  timestamp: 1649412882
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b2d55_00001
  
R

Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_b2d55_00000,RUNNING,192.168.29.18:45676,0.00103179,6,8,3,3.48598,0.692401,0.32
objective_b2d55_00002,RUNNING,192.168.29.18:45677,0.119756,13,9,1,2.86,0.670784,0.32
objective_b2d55_00003,RUNNING,192.168.29.18:45672,0.000468887,6,3,,,,
objective_b2d55_00004,RUNNING,192.168.29.18:45678,0.00809948,16,6,,,,
objective_b2d55_00005,RUNNING,192.168.29.18:45680,0.268566,9,10,,,,
objective_b2d55_00006,RUNNING,192.168.29.18:45673,0.230558,6,7,,,,
objective_b2d55_00007,RUNNING,192.168.29.18:45669,1.31755,2,10,,,,
objective_b2d55_00008,RUNNING,192.168.29.18:45671,0.0106242,12,9,1,1.8778,0.69098,0.32
objective_b2d55_00009,RUNNING,192.168.29.18:45679,0.0208905,10,10,,,,
objective_b2d55_00010,RUNNING,192.168.29.18:45675,0.0210939,14,8,,,,


Result for objective_b2d55_00011:
  date: 2022-04-08_15-44-44
  done: false
  eval-error: 0.32
  eval-logloss: 0.61732
  experiment_id: 76dbe81edb0d4e2dab261020cf018c49
  hostname: home
  iterations_since_restore: 1
  node_ip: 192.168.29.18
  pid: 45670
  should_checkpoint: true
  time_since_restore: 2.7328004837036133
  time_this_iter_s: 2.7328004837036133
  time_total_s: 2.7328004837036133
  timestamp: 1649412884
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b2d55_00011
  
Result for objective_b2d55_00007:
  date: 2022-04-08_15-44-45
  done: false
  eval-error: 0.32
  eval-logloss: 0.666148
  experiment_id: c0fd06f3160e4a2eb8ca509cbacbc2fe
  hostname: home
  iterations_since_restore: 1
  node_ip: 192.168.29.18
  pid: 45669
  should_checkpoint: true
  time_since_restore: 2.599325656890869
  time_this_iter_s: 2.599325656890869
  time_total_s: 2.599325656890869
  timestamp: 1649412885
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b2d55_00007
  
Resu

...

Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_b2d55_00088,RUNNING,192.168.29.18:49318,0.365494,15,3,6,14.9217,0.451985,0.26
objective_b2d55_00089,RUNNING,192.168.29.18:49320,0.565945,11,3,7,15.5133,0.433517,0.26
objective_b2d55_00094,RUNNING,192.168.29.18:49628,1.07169,6,2,2,4.98335,0.501638,0.22
objective_b2d55_00097,RUNNING,192.168.29.18:49820,0.00140936,4,6,,,,
objective_b2d55_00098,RUNNING,192.168.29.18:49900,0.00126109,17,6,,,,
objective_b2d55_00099,PENDING,,0.0474628,9,3,,,,
objective_b2d55_00000,TERMINATED,192.168.29.18:45676,0.00103179,6,8,8,15.4837,0.69117,0.32
objective_b2d55_00001,TERMINATED,192.168.29.18:45674,1.82752,5,3,1,1.74782,0.701446,0.34
objective_b2d55_00002,TERMINATED,192.168.29.18:45677,0.119756,13,9,10,18.7642,0.531971,0.2
objective_b2d55_00003,TERMINATED,192.168.29.18:45672,0.000468887,6,3,1,4.63458,0.693,0.34


Result for objective_b2d55_00097:
  date: 2022-04-08_15-46-33
  done: true
  eval-error: 0.32
  eval-logloss: 0.692881
  experiment_id: d302f3405e774291b72a8aa4596df137
  hostname: home
  iterations_since_restore: 1
  node_ip: 192.168.29.18
  pid: 49820
  should_checkpoint: true
  time_since_restore: 1.4331727027893066
  time_this_iter_s: 1.4331727027893066
  time_total_s: 1.4331727027893066
  timestamp: 1649412993
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b2d55_00097
  
Result for objective_b2d55_00094:
  date: 2022-04-08_15-46-34
  done: true
  eval-error: 0.16
  eval-logloss: 0.537664
  experiment_id: 8f796b7a6bcd46498db27d9089575900
  hostname: home
  iterations_since_restore: 4
  node_ip: 192.168.29.18
  pid: 49628
  time_since_restore: 7.3300065994262695
  time_this_iter_s: 1.2135677337646484
  time_total_s: 7.3300065994262695
  timestamp: 1649412994
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: b2d55_00094
  
Result for objective_b2d55_0

Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_b2d55_00000,TERMINATED,192.168.29.18:45676,0.00103179,6,8,8,15.4837,0.69117,0.32
objective_b2d55_00001,TERMINATED,192.168.29.18:45674,1.82752,5,3,1,1.74782,0.701446,0.34
objective_b2d55_00002,TERMINATED,192.168.29.18:45677,0.119756,13,9,10,18.7642,0.531971,0.2
objective_b2d55_00003,TERMINATED,192.168.29.18:45672,0.000468887,6,3,1,4.63458,0.693,0.34
objective_b2d55_00004,TERMINATED,192.168.29.18:45678,0.00809948,16,6,1,3.43367,0.691628,0.32
objective_b2d55_00005,TERMINATED,192.168.29.18:45680,0.268566,9,10,10,18.0194,0.496568,0.24
objective_b2d55_00006,TERMINATED,192.168.29.18:45673,0.230558,6,7,10,21.2498,0.450623,0.2
objective_b2d55_00007,TERMINATED,192.168.29.18:45669,1.31755,2,10,2,4.28275,0.775961,0.38
objective_b2d55_00008,TERMINATED,192.168.29.18:45671,0.0106242,12,9,4,8.39657,0.683742,0.28
objective_b2d55_00009,TERMINATED,192.168.29.18:45679,0.0208905,10,10,1,3.45215,0.688584,0.32


2022-04-08 15:46:35,617	INFO tune.py:639 -- Total run time: 116.76 seconds (116.23 seconds for the tuning loop).


Best model parameters: {'objective': 'binary:logistic', 'learning_rate': 0.35500968729171256, 'eval_metric': ['logloss', 'error'], 'max_depth': 17, 'min_child_weight': 1}
Best model total accuracy: 0.8600


In [6]:
# Reading the dataset:
df = pd.read_csv('/home/vivek/Project/new/Zernike_Moments_YN_250.csv', header=None)

In [7]:
# Shuffling the whole dataset:
df = df.sample(frac=1, random_state=0).reset_index(drop=True)

In [8]:
# Converting categorical values of Target feature into numerical:
df[289].replace(['YES','NO'], [1,0], inplace=True)

In [9]:
# independent and dependent features:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [10]:
# Train-Test spilt
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
# Using Best parameters found from HyperParameter Optimization done using Optuna:
xgb = XGBClassifier(
                    learning_rate = 0.3295,
                    max_depth = 12,
                    min_child_weight = 4
                   )

In [12]:
# Fitting the Training data:
xgb.fit(x_train, y_train)

/home/vivek/vivek/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:47:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.3295,
              max_delta_step=0, max_depth=12, min_child_weight=4, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
# Predicting the test data:
y_pred = xgb.predict(x_test)

### Results:

In [14]:
print("Accuracy :", round(accuracy_score(y_test, y_pred), 2)*100, "%")

Accuracy : 84.0 %


In [15]:
print("Precision :", round(precision_score(y_test, y_pred), 2)*100, "%")

Precision : 87.0 %


In [16]:
print("Recall :", round(recall_score(y_test, y_pred), 2)*100, "%")

Recall : 87.0 %


In [17]:
print("f1_Score :", round(f1_score(y_test, y_pred), 2)*100, "%")

f1_Score : 87.0 %


In [18]:
print("Confusion Matrix :")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix :
[[15  4]
 [ 4 27]]


In [19]:
print("Classification Report :")
print(classification_report(y_test, y_pred))

Classification Report :
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        19
           1       0.87      0.87      0.87        31

    accuracy                           0.84        50
   macro avg       0.83      0.83      0.83        50
weighted avg       0.84      0.84      0.84        50

